In [1]:
pip install notebook

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

import time
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from requests import post, get
import base64

In [183]:
df = pd.read_csv("df.csv")

In [179]:
data = pd.read_csv("data.csv")
artist = pd.read_csv("data_by_artist.csv")
genre = pd.read_csv("data_by_genres.csv")
year = pd.read_csv("data_by_year.csv")
genres = pd.read_csv("data_w_genres.csv")

track的音訊變數的解釋：
https://developer.spotify.com/documentation/web-api/reference/get-audio-features


額外的音訊分析變數：
https://developer.spotify.com/documentation/web-api/reference/get-audio-analysis

## 蒐集Spotify歌曲資料的音樂類別資訊（genres）

In [2]:
client_id='c91a738c12e8443bb4915645c20a9599'
client_secret='da0d247a355e4abc9d3130ed70c01ff1'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,client_secret=client_secret))

In [ ]:
start_index = 0
end_index = 134001
batch_size = 50

for i in range(start_index, end_index, batch_size):
    # 取得要處理的 track_ids
    track_ids = df.iloc[i:i+batch_size]['id'].tolist()

    # 使用 Spotify API 取得 tracks 的詳細資訊
    tracks_info = sp.tracks(track_ids)

    # 建立一個空的 DataFrame 來存儲結果
    result_df = pd.DataFrame(columns=['id', 'genres'])

    time.sleep(0.3) 
    # 遍歷每個 track 的詳細資訊
    for track_info in tracks_info['tracks']:
        artists = track_info['artists']
        artist_ids = [artist['id'] for artist in artists]
        
        # 使用 Spotify API 一次性獲取所有藝術家的詳細資訊
        artists_info = sp.artists(artist_ids)

        # 提取每個藝術家的音樂流派並存儲到 genres_list 中
        genres_list = []
        for artist_info in artists_info['artists']:
            genres_list.extend(artist_info['genres'])

        # 將 track 的 ID 和音樂流派資訊存儲到 result_df 中
        result_df = pd.concat([result_df, pd.DataFrame({'id': [track_info['id']], 'genres': [genres_list]})], ignore_index=True)
        time.sleep(0.5) 

    # 將結果合併到原始 DataFrame 中
    df.iloc[i:i+batch_size, df.columns.get_loc('genres')] = result_df['genres']

In [266]:
df.to_csv('df.csv', index=False) 

### 用先前已蒐集的歌曲資料進行整理

In [259]:
start = 147001
end = 149001

for i in range(start, end):
    target = df.iloc[i]['artists']
    # 去除特殊符號 []
    target = target.strip("[]").replace("'", "").split(", ")
    # 找出完全符合 target 的索引
    index = df[df['artists'].apply(lambda x: x.strip("[]").replace("'", "").split(", ") == target)]
    if not index.empty:
        if i > index.index[0]:  # i 大於 index
            df.at[i, 'genres'] = index.iloc[0]['genres']

### 補齊遺漏未蒐集資料的Genres

In [172]:
na_data = df[(df['genres'].isnull()) & (df.index >= start) & (df.index < end)].index

In [173]:
track_ids = df.iloc[na_data]['id'].tolist()

In [174]:
result_df = pd.DataFrame(columns=['id', 'genres'])

# 分批處理每一組 50 筆資料
for i in range(0, len(na_data), 50):
    # 取得這一組的 track_ids
    batch_track_ids = track_ids[i:i+50]
    
    # 使用 Spotify API 獲取這一組 track 的詳細資訊
    tracks_info = sp.tracks(batch_track_ids)
    
    # 初始化存儲每一組結果的 DataFrame
    batch_result_df = pd.DataFrame(columns=['id', 'genres'])
    
    # 遍歷每個 track 的詳細資訊
    for track_info in tracks_info['tracks']:
        artists = track_info['artists']
        artist_ids = [artist['id'] for artist in artists]

        # 使用 Spotify API 獲取每個藝術家的詳細資訊
        artists_info = sp.artists(artist_ids)

        # 提取每個藝術家的音樂流派並存儲到 genres_list 中
        genres_set = set()  # 使用集合來去除重複的值
        for artist_info in artists_info['artists']:
            genres_set.update(artist_info['genres'])  # 使用 update 方法將所有流派加入集合中

        # 將集合轉換為列表，並將其存入 batch_result_df 中
        genres_list = list(genres_set)
    
        # 將 track 的 ID 和音樂流派資訊存儲到 batch_result_df 中
        batch_result_df = pd.concat([batch_result_df, pd.DataFrame({'id': [track_info['id']], 'genres': [genres_list]})], ignore_index=True)
    # 將這一組結果存入 result_df 中
    result_df = pd.concat([result_df, batch_result_df], ignore_index=True)

# 將結果更新到原始 DataFrame 中
df.iloc[na_data, df.columns.get_loc('genres')] = result_df['genres']

In [187]:
df.iloc[na_data, df.columns.get_loc('genres')] = result_df['genres']